In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from graphdatascience import GraphDataScience
from neo4j_tools import gds_db_load, gds_utils

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

In [2]:
gds_db_load.__file__

'/opt/conda/lib/python3.11/site-packages/neo4j_tools/gds_db_load.py'

In [3]:
from dotenv import load_dotenv
import os

# Neo4j
NEO4J_URI = 'bolt://database:7687' #change this
NEO4J_PASSWORD = 'password' #change this
NEO4J_USERNAME = 'neo4j'
AURA_DS = False

In [4]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=AURA_DS)

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


In [5]:
gds.debug.sysInfo()

,key,value
0,gdsVersion,2.6.8
1,gdsEdition,Unlicensed
2,neo4jVersion,5.19.0
3,minimumRequiredJavaVersion,11
4,unavailableCompatibility,Neo4j 4.4
...,...,...
98,server.memory.pagecache.size,536870912
99,server.memory.off_heap.transaction_max_size,2147483648
100,dbms.memory.transaction.total.max,2293025996
101,db.memory.transaction.total.max,0


In [6]:
department_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/department.csv')

In [7]:
department_df

,departmentNo,departmentName,sectionNo,sectionName
0,1676,Jersey Basic,16,Womens Everyday Basics
1,1339,Clean Lingerie,61,Womens Lingerie
2,3608,Tights basic,62,"Womens Nightwear, Socks & Tigh"
3,5883,Jersey Basic,26,Men Underwear
4,2032,Jersey,8,Mama
...,...,...,...,...
261,7510,Woven,28,Men Edition
262,3420,Small Accessories Extended,66,Womens Small accessories
263,5231,Jacket,31,Mens Outerwear
264,8090,Promotion/Other/Offer,29,Men Other


In [9]:
%%time

import pandas as pd
from neo4j_tools import gds_db_load

# get source data - it has been pre-formatted. If you would like to re-generate from source on Kaggle, see the data-prep.ipynb notebook
department_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/department.csv')
product_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/product.csv')
article_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/article.csv')
customer_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/customer.csv')
transaction_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/transaction.csv')

# create constraints - one uniqueness constraint for each node label
gds.run_cypher('CREATE CONSTRAINT unique_department_no IF NOT EXISTS FOR (n:Department) REQUIRE n.departmentNo IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_product_code IF NOT EXISTS FOR (n:Product) REQUIRE n.productCode IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_article_id IF NOT EXISTS FOR (n:Article) REQUIRE n.articleId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_customer_id IF NOT EXISTS FOR (n:Customer) REQUIRE n.customerId IS UNIQUE')

# load nodes
gds_db_load.load_nodes(gds, department_df, 'departmentNo', 'Department')
gds_db_load.load_nodes(gds, article_df.drop(columns=['productCode', 'departmentNo']), 'articleId', 'Article')
gds_db_load.load_nodes(gds, product_df, 'productCode', 'Product')
gds_db_load.load_nodes(gds, customer_df, 'customerId', 'Customer')

# load relationships
gds_db_load.load_rels(gds, article_df[['articleId', 'departmentNo']], source_target_labels=('Article', 'Department'),
                      source_node_key='articleId', target_node_key='departmentNo',
                      rel_type='FROM_DEPARTMENT')
gds_db_load.load_rels(gds, article_df[['articleId', 'productCode']], source_target_labels=('Article', 'Product'),
                      source_node_key='articleId',target_node_key='productCode',
                      rel_type='VARIANT_OF')
gds_db_load.load_rels(gds, transaction_df, source_target_labels=('Customer', 'Article'),
                      source_node_key='customerId', target_node_key='articleId', rel_key='txId',
                      rel_type='PURCHASED')

# convert transaction dates
gds.run_cypher('''
MATCH (:Customer)-[r:PURCHASED]->()
SET r.tDat = date(r.tDat)
''')

# create combined text property. This will help simplify later with semantic search and RAG
gds.run_cypher("""
    MATCH(p:Product)
    SET p.text = '##Product\n' +
        'Name: ' + p.prodName + '\n' +
        'Type: ' + p.productTypeName + '\n' +
        'Group: ' + p.productGroupName + '\n' +
        'Garment Type: ' + p.garmentGroupName + '\n' +
        'Description: ' + p.detailDesc
    RETURN count(p) AS propertySetCount
    """)

======  loading Department nodes  ======
staging 266 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Department {departmentNo: rec.departmentNo})
SET n.departmentName = rec.departmentName, n.sectionNo = rec.sectionNo, n.sectionName = rec.sectionName
RETURN count(n) AS nodeLoadedCount
```

Loaded 266 of 266 nodes
======  loading Article nodes  ======
staging 13,351 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Article {articleId: rec.articleId})
SET n.prodName = rec.prodName, n.productTypeName = rec.productTypeName, n.graphicalAppearanceNo = rec.graphicalAppearanceNo, n.graphicalAppearanceName = rec.graphicalAppearanceName, n.colourGroupCode = rec.colourGroupCode, n.colourGroupName = rec.colourGroupName
RETURN count(n) AS nodeLoadedCount
```

Loaded 10,000 of 13,351 nodes
Loaded 13,351 of 13,351 nodes
======  loading Product nodes  ======
staging 8,044 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Product {productCode: rec.produc

,propertySetCount
0,8044


In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Alibaba-NLP/gte-base-en-v1.5"
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"trust_remote_code":True}
)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

product_emb_df = product_df[['productCode', 'prodName', 'productTypeName', 'productGroupName', 'garmentGroupName', 'detailDesc']]
product_emb_df = product_emb_df[product_emb_df.detailDesc.notnull()]

def create_doc(row):
    return f'''
##Product
Name: {row.prodName}
Type: {row.productTypeName}
Group: {row.productGroupName}
Garment Type: {row.garmentGroupName}
Description: {row.detailDesc}
'''

product_emb_df['text'] = product_emb_df.apply(create_doc, axis=1)
product_emb_df = product_emb_df.drop(columns=['prodName', 'productTypeName', 'productGroupName', 'garmentGroupName', 'detailDesc'])
product_emb_df

,productCode,text
0,108775,\n##Product\nName: Strap top\nType: Vest top\nGroup: Garment Upper body\nGarment Type: Jersey Basic\nDescription: Jersey top with narrow shoulder straps.\n
1,110065,"\n##Product\nName: OP T-shirt (Idro)\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Microfibre T-shirt bra with underwired, moulded, lightly padded cups that shape the bust and provide good support. Narrow adjustable shoulder straps and a narrow hook-and-eye fastening at the back. Without visible seams for greater comfort.\n"
2,111565,"\n##Product\nName: 20 den 1p Stockings\nType: Underwear Tights\nGroup: Socks & Tights\nGarment Type: Socks and Tights\nDescription: Semi shiny nylon stockings with a wide, reinforced trim at the top. Use with a suspender belt. 20 denier.\n"
3,111586,\n##Product\nName: Shape Up 30 den 1p Tights\nType: Leggings/Tights\nGroup: Garment Lower body\nGarment Type: Socks and Tights\nDescription: Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.\n
4,111593,"\n##Product\nName: Support 40 den 1p Tights\nType: Underwear Tights\nGroup: Socks & Tights\nGarment Type: Socks and Tights\nDescription: Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.\n"
...,...,...
8039,936862,"\n##Product\nName: EDC Marla dress\nType: Dress\nGroup: Garment Full body\nGarment Type: Special Offers\nDescription: Calf-length dress in a patterned Tencel™ lyocell weave with a V-neck, sewn in wrapover at the top and decorative ties at one side. 3/4-length dolman sleeves with narrow, covered elastication at the cuffs. Gathered seam at the waist with concealed elastication and a flared skirt with a gathered tier at the hem for added width. Unlined.\n"
8040,936979,\n##Product\nName: Class Filippa Necklace\nType: Necklace\nGroup: Accessories\nGarment Type: Accessories\nDescription: Metal chain necklace with a pendant. Adjustable length.\n
8041,937138,\n##Product\nName: Flirty Albin bracelet pk\nType: Bracelet\nGroup: Accessories\nGarment Type: Accessories\nDescription: Metal chain bracelets. Two plain and two with pendants. Adjustable length.\n
8042,942187,"\n##Product\nName: ED Sasha tee\nType: T-shirt\nGroup: Garment Upper body\nGarment Type: Jersey Fancy\nDescription: Oversized, straight-cut T-shirt in a soft modal and cotton jersey blend with a ribbed neckline and low dropped shoulders.\n"


In [16]:
%%time

count = 0
embeddings = []
for docs in gds_db_load.chunks(product_emb_df.text, n=500):
    count += len(docs)
    print(f'Embedded {count} of {product_emb_df.shape[0]}')
    embeddings.extend(embedding_model.embed_documents(docs))

# Set as column of dataframe to prepare for loading
product_emb_df['textEmbedding'] = embeddings
product_emb_df

Embedded 500 of 8018
Embedded 1000 of 8018
Embedded 1500 of 8018
Embedded 2000 of 8018
Embedded 2500 of 8018
Embedded 3000 of 8018
Embedded 3500 of 8018
Embedded 4000 of 8018
Embedded 4500 of 8018
Embedded 5000 of 8018
Embedded 5500 of 8018
Embedded 6000 of 8018
Embedded 6500 of 8018
Embedded 7000 of 8018
Embedded 7500 of 8018
Embedded 8000 of 8018
Embedded 8018 of 8018
CPU times: user 2h 1min 50s, sys: 3min 33s, total: 2h 5min 23s
Wall time: 21min 24s


,productCode,text,textEmbedding
0,108775,\n##Product\nName: Strap top\nType: Vest top\nGroup: Garment Upper body\nGarment Type: Jersey Basic\nDescription: Jersey top with narrow shoulder straps.\n,"[-0.10236894339323044, -0.7117778658866882, 0.2316431999206543, -0.795796275138855, 2.3781936168670654, 0.33821821212768555, -0.541305661201477, -1.2975690364837646, -0.32866671681404114, -0.6778296232223511, 0.09256365150213242, -1.5653389692306519, -0.10766911506652832, -0.7279382944107056, -1.2005813121795654, 0.6615098714828491, -0.17001259326934814, 1.3723795413970947, 0.718006432056427, -0.619225025177002, -0.43622055649757385, 0.21899406611919403, -0.05707612261176109, 0.4582121968269..."
1,110065,"\n##Product\nName: OP T-shirt (Idro)\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Microfibre T-shirt bra with underwired, moulded, lightly padded cups that shape the bust and provide good support. Narrow adjustable shoulder straps and a narrow hook-and-eye fastening at the back. Without visible seams for greater comfort.\n","[0.8598872423171997, -0.5367504358291626, -0.578113853931427, -1.3442884683609009, -0.006047095637768507, -1.035750150680542, 0.006694616749882698, -0.11394979059696198, -0.6393452882766724, -0.3535594046115875, -0.12346799671649933, -1.7035826444625854, -0.20806844532489777, -1.0102680921554565, -1.0189533233642578, 0.2569650411605835, -0.7216237187385559, -0.7838944792747498, 0.5140393376350403, -0.3973971903324127, 0.027540750801563263, 0.17806978523731232, 0.44973650574684143, 0.64569973..."
2,111565,"\n##Product\nName: 20 den 1p Stockings\nType: Underwear Tights\nGroup: Socks & Tights\nGarment Type: Socks and Tights\nDescription: Semi shiny nylon stockings with a wide, reinforced trim at the top. Use with a suspender belt. 20 denier.\n","[-0.5412996411323547, -0.4270375072956085, 0.13645537197589874, -1.8090810775756836, 0.798502504825592, -0.5747131705284119, -1.0101453065872192, -0.38747525215148926, 0.251229852437973, 0.1383482664823532, -0.8225523829460144, -0.6079437136650085, 0.7127873301506042, 0.41398996114730835, 0.26940590143203735, 0.8674864768981934, -0.8350976705551147, 0.4913423955440521, 1.6412607431411743, -0.6587619781494141, 0.10748078674077988, -0.17043517529964447, -0.4879412055015564, 0.5867806673049927,..."
3,111586,\n##Product\nName: Shape Up 30 den 1p Tights\nType: Leggings/Tights\nGroup: Garment Lower body\nGarment Type: Socks and Tights\nDescription: Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.\n,"[0.16514360904693604, -0.5562731623649597, -1.0193394422531128, -1.6503777503967285, 0.5747876763343811, -0.08356606215238571, -0.6986817121505737, -0.19797205924987793, -0.4415811002254486, -0.3361862897872925, -0.32900476455688477, -0.40009385347366333, 0.6294813752174377, 0.678534209728241, -1.153185248374939, -0.05050051584839821, -0.7512076497077942, 0.020304208621382713, 1.6743091344833374, -1.4841333627700806, -0.031520355492830276, 0.04262666031718254, -0.8259020447731018, 0.66383701..."
4,111593,"\n##Product\nName: Support 40 den 1p Tights\nType: Underwear Tights\nGroup: Socks & Tights\nGarment Type: Socks and Tights\nDescription: Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.\n","[0.22089828550815582, 0.38761723041534424, -1.0561327934265137, -1.119616150856018, 0.860040545463562, -0.5558141469955444, -0.719001293182373, -0.2529374361038208, -0.322311133146286, 0.2358253449201584, -0.4837040901184082, -0.05121878534555435, 0.3689073622226715, 0.5951403379440308, 0.33235567808151245, 0.05891694128513336, -0.47190120816230774, -0.33969569206237793, 1.5433435440063477, -0.4647504985332489, -0.08965236693620682, 0.24993690848350525, -0.3494814932346344, 1.025807023048400..."
...,...,...,...
8039,936862,"\n##Product\nName: EDC Marla dress\nType: Dress\nGroup: Garment Full body\nGarment Type: Special Offers\nDescriptio

In [17]:
embedding_dimension = 768

In [18]:
# load vector properties
records = product_emb_df[['productCode', 'textEmbedding']].to_dict('records')
print(f'======  loading Product text embeddings ======')
total = len(records)
print(f'staging {total:,} records')
cumulative_count = 0
for recs in gds_db_load.chunks(records, n=100):
    res = gds.run_cypher('''
    UNWIND $recs AS rec
    MATCH(n:Product {productCode: rec.productCode})
    CALL db.create.setNodeVectorProperty(n, "textEmbedding", rec.textEmbedding)
    RETURN count(n) AS propertySetCount
    ''', params={'recs': recs})
    cumulative_count += res.iloc[0, 0]
    print(f'Set {cumulative_count:,} of {total:,} text embeddings')

#create index
gds.run_cypher('''
CREATE VECTOR INDEX product_text_embeddings IF NOT EXISTS FOR (n:Product) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dim),
 `vector.similarity_function`: 'cosine'
}}''', params={'dim': embedding_dimension})

gds.run_cypher('CALL db.awaitIndex("product_text_embeddings", 300)')

======  loading Product text embeddings ======
staging 8,018 records
Set 100 of 8,018 text embeddings
Set 200 of 8,018 text embeddings
Set 300 of 8,018 text embeddings
Set 400 of 8,018 text embeddings
Set 500 of 8,018 text embeddings
Set 600 of 8,018 text embeddings
Set 700 of 8,018 text embeddings
Set 800 of 8,018 text embeddings
Set 900 of 8,018 text embeddings
Set 1,000 of 8,018 text embeddings
Set 1,100 of 8,018 text embeddings
Set 1,200 of 8,018 text embeddings
Set 1,300 of 8,018 text embeddings
Set 1,400 of 8,018 text embeddings
Set 1,500 of 8,018 text embeddings
Set 1,600 of 8,018 text embeddings
Set 1,700 of 8,018 text embeddings
Set 1,800 of 8,018 text embeddings
Set 1,900 of 8,018 text embeddings
Set 2,000 of 8,018 text embeddings
Set 2,100 of 8,018 text embeddings
Set 2,200 of 8,018 text embeddings
Set 2,300 of 8,018 text embeddings
Set 2,400 of 8,018 text embeddings
Set 2,500 of 8,018 text embeddings
Set 2,600 of 8,018 text embeddings
Set 2,700 of 8,018 text embeddings
Set 

""


In [27]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = 'black lingeries'

query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

query vector length: 768
query vector sample: [0.060306910425424576, -1.1521093845367432, -0.1058843582868576, -0.3318834900856018, 0.2584519386291504, -0.07973887771368027, 0.8409785628318787, -0.44283315539360046, 0.7798233032226562, 1.3583407402038574]


In [28]:
gds.run_cypher('''
CALL db.index.vector.queryNodes("product_text_embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.productCode AS productCode,
    product.text AS text,
    score
''', params={'queryVector': query_vector})

,productCode,text,score
0,748578,"##Product\nName: Khloe SP Bralette olympus\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Strapless balconette bralette in lace with underwired, thickly padded cups to maximise the bust and create a fuller cleavage. Detachable, adjustable shoulder straps, side support and a wide fastening at the back with several pairs of hooks and eyes.",0.793288
1,781042,"##Product\nName: Beast SP Bralette\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Bralette in lace and mesh with underwired, thickly padded cups to maximise the bust and create a fuller cleavage. Narrow, adjustable shoulder straps and a hook-and-eye fastening at the back.",0.793102
2,798418,"##Product\nName: Kendrick SP bralette\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Super push-up bralette in lace and mesh with underwired, thickly padded cups to maximise the bust and create a fuller cleavage. Narrow, adjustable shoulder straps, a wide lace trim at the hem and a hook-and-eye fastening at the back.",0.790458
3,656345,"##Product\nName: Atlanta Bralette Valencia\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Push-up lace bralette with underwired, moulded, padded cups for a larger bust and fuller cleavage. Narrow, adjustable shoulder straps, a lace trim at the hem and a wide hook-and-eye fastening.",0.788625
4,606679,"##Product\nName: Angela cheeky bk lace\nType: Underwear bottom\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Lace hipster briefs with a lined gusset, wide sides and cutaway coverage at the back.",0.787398
5,697426,"##Product\nName: Carmenzita strapless D-F\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Balconette bra in microfibre and lace with underwired, padded cups that lift and shape the bust and provide extra-firm support. Detachable, adjustable shoulder straps and a hook-and-eye fastening at the back.",0.785491
6,766965,"##Product\nName: Glam body Shay followup\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Body in glittery jersey with lightly padded cups to shape the bust and provide good support, and a low-cut back. Adjustable shoulder straps, high-cut legs and a lined gusset.",0.784034
7,505882,"##Product\nName: Kelly Strapless (Sofia) 2pk\nType: Unknown\nGroup: Unknown\nGarment Type: Under-, Nightwear\nDescription: Strapless balconette bras in a soft cotton blend with lace details and underwired, padded cups that lift and shape. Silicone trim at the hem and a fastening at the back with three pairs of hooks and eyes.",0.783014
8,684087,"##Product\nName: OPEN (softbra lace)\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Soft, non-wired lace bralettes with mesh-lined, triangular cups that give the bust a natural shape and provide light support. Wide shoulder straps, a lace trim at the hem and a wide fastening at the back with three pairs of hooks and eyes.",0.781977
9,526980,"##Product\nName: Marissa Hoodie\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Long hooded top in velour with embroidered text on the front, long raglan sleeves, a kangaroo pocket and ribbing at the cuffs and hem.",0.781785


In [32]:
from langchain.vectorstores.neo4j_vector import Neo4jVector

kg_vector_search = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name='product_text_embeddings')

In [33]:
res = kg_vector_search.similarity_search(search_prompt, k=10)
res

[Document(page_content='##Product\nName: Khloe SP Bralette olympus\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Nightwear\nDescription: Strapless balconette bralette in lace with underwired, thickly padded cups to maximise the bust and create a fuller cleavage. Detachable, adjustable shoulder straps, side support and a wide fastening at the back with several pairs of hooks and eyes.', metadata={'garmentGroupName': 'Under-, Nightwear', 'prodName': 'Khloe SP Bralette olympus', 'garmentGroupNo': 1017, 'productCode': 748578, 'productTypeName': 'Bra', 'productTypeNo': 306, 'detailDesc': 'Strapless balconette bralette in lace with underwired, thickly padded cups to maximise the bust and create a fuller cleavage. Detachable, adjustable shoulder straps, side support and a wide fastening at the back with several pairs of hooks and eyes.', 'productGroupName': 'Underwear'}),
 Document(page_content='##Product\nName: Beast SP Bralette\nType: Bra\nGroup: Underwear\nGarment Type: Under-, Night

In [34]:
res = kg_vector_search.similarity_search('halter neck top', k=10)
pd.DataFrame([{'document': d.page_content} for d in res])

,document
0,"##Product\nName: Elsa halter neck top\nType: Top\nGroup: Garment Upper body\nGarment Type: Special Offers\nDescription: Slightly shorter halterneck top in a patterned weave with decorative ties at the back of the neck and an opening at the front. Seam under the bust with a drawstring that ties at the sides, and a frill at the hem with slits in the sides. Unlined."
1,##Product\nName: Halter\nType: Vest top\nGroup: Garment Upper body\nGarment Type: Jersey Basic\nDescription: Gently flared vest top in jersey with a sheen. Narrow shoulder straps that cross at the back. Lined at the top.
2,##Product\nName: Cavalier top\nType: Top\nGroup: Garment Upper body\nGarment Type: Dresses Ladies\nDescription: Sleeveless lace halterneck top with narrow shoulder straps that are lace-covered at the back and fasten with narrow ties at the back of the neck. Jersey lining at the front.
3,"##Product\nName: Adele Blouse CK\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Knitwear\nDescription: Shorter-style top in an airy rib knit. Deep wide neckline with an opening and ties at the front, long raglan sleeves and frill-trimmed smocking at the cuffs and hem."
4,"##Product\nName: RR 32 2 BLOUSE high neck\nType: Blouse\nGroup: Garment Upper body\nGarment Type: Unknown\nDescription: Long-sleeved blouse in a crêpe weave with a small frilled collar, an opening with ties at the back of the neck and elasticated cuffs with a flounced trim."
5,"##Product\nName: CS Dunce Top\nType: Top\nGroup: Garment Upper body\nGarment Type: Blouses\nDescription: Shorter-style blouse in woven fabric with a small, gathered stand-up collar, an opening at the back and a hook-and-eye fastening at the back of the neck. Long puff sleeves and wide cuffs with covered buttons. Seam with wide, gathered ties at the hem that fasten at the back. Unlined."
6,"##Product\nName: Elyse\nType: Top\nGroup: Garment Upper body\nGarment Type: Blouses\nDescription: Blouse in woven fabric with ties at the back of the neck, stand-up collar and concealed zip at the back. Long sleeves with buttoned cuffs and a seam at the waist and flare to the hem with slits in the sides."
7,"##Product\nName: Belinda Top\nType: Blouse\nGroup: Garment Upper body\nGarment Type: Blouses\nDescription: Sleeveless blouse in a softly draping weave with pin-tucks down the front. Small stand-up collar, concealed buttons at the front, a yoke with a pleat at the back, and a gently rounded hem. Unlined."
8,##Product\nName: Waterloo\nType: Vest top\nGroup: Garment Upper body\nGarment Type: Blouses\nDescription: Sleeveless blouse in satin with a draped neckline at the front and a concealed zip at the back of the neck.
9,"##Product\nName: Akita lace top\nType: Blouse\nGroup: Garment Upper body\nGarment Type: Blouses\nDescription: Wrapover blouse in an airy cotton weave with broderie anglaise. V-neck, concealed press-studs and ties at one side. Short, wide sleeves with a scalloped trim, and a scalloped neckline. Unlined."
